In [1]:
using Plots: Plot, plot, plot!
using LinearAlgebra: norm, I, eigvals, dot
using Printf

In [7]:
function ArmijoWolfeLineSearch(
        f,
        x::AbstractArray,
        p::AbstractArray;
        αinit::Real=1,
        τ::Real=1.1,
        c1::Real=1e-4,
        c2::Real=0.9,
        ϵα::Real=1e-16,
        ϵgrad::Real=1e-12,
        safeguard::Real=0.20,
        MaxEvaluations::Integer=1000
    )

    ϕ = (α) -> begin
        v, _ = f(x + α * p)
        return v
    end

    ϕd = (α) -> begin
        _, gradient = f(x + α * p)
        return dot(p, gradient)
    end

    α = αinit
    local αgrad

    ϕ_0 = ϕ(0)
    ϕd_0 = ϕd(0)

    while MaxEvaluations > 0
        αcurr = ϕ(α)
        αgrad = ϕd(α)
        MaxEvaluations -= 1

        if (αcurr ≤ ϕ_0 + c1 * α * ϕd_0) && (abs(αgrad) ≤ -c2 * ϕd_0)
            return α
        end
        
        if αgrad ≥ 0
            break
        end
        α *= τ
    end

    αlo = 0
    αhi = α
    αlograd = ϕd_0
    αhigrad = αgrad

    while (MaxEvaluations > 0) && (αhi - αlo) > ϵα && (αgrad > ϵgrad)
        α = (αlo * αhigrad - αhi * αlograd)/(αhigrad - αlograd)
        α = max(
            αlo + (αhi - αlo) * safeguard,
            min(αhi - (αhi - αlo) * safeguard, α)
        )

        αcurr = ϕ(α)
        αgrad = ϕd(α)
        MaxEvaluations -= 1

        if (αcurr ≤ ϕ_0 + c1 * α * ϕd_0) && (abs(αgrad) ≤ -c2 * ϕd_0)
            break
        end

        if αgrad < 0
            αlo = α
            αlograd = αgrad
        else
            αhi = α
            if αhi ≤ ϵα
                break
            end
            αhigrad = αgrad
        end
    end

    return α
end

ArmijoWolfeLineSearch (generic function with 1 method)

In [12]:
function BFGS2(f;
        x::Union{Nothing, Vector}=nothing,
        ϵ::Real=1e-6
    )
    if isnothing(x)
        _, x = f(nothing)
    end

    k = 0
    _, gradient = f(x)
    H = I

    while norm(gradient) > ϵ
        # direction from the inverse
        p = -H * gradient
        # direction by inverting the hessian
        #p = H \ -gradient

        α = ArmijoWolfeLineSearch(f, x, p)

        previousx = x
        x = x + α * p

        previousgradient = gradient
        _, gradient = f(x)

        s = x - previousx
        y = gradient - previousgradient

        curvature = dot(y, s)

        ## dampening
        θ = if (curvature ≥ 0.2 * s' * H * s)
            1
        else
            (0.8 * s' * H * s)/(s' * H * s - curvature)
        end
        r = θ * y + (1 - θ) * H * s
        

        ρ = 1 / curvature

        # finding already the inverse
        #H = (I - ρ * s * y') * H * (I - ρ * y * s') + ρ * s * s'
        #H = H + ρ * ((1 + ρ * y' * H * y) * (s * s') - H * y * s' - (H * y * s')')
        
        # finding only the approximated hessian
        #H = H - (H * s * s' * H)/(s' * H * s) + (y * y')/(y' * s)

        # dampened hessian
        #H = H - (H * s * s' * H)/(s' * H * s) + (r * r')/(r' * s)

        # inverse dampened hessian
        H = (I - ρ * s * r') * H * (I - ρ * r * s') + ρ * s * s'
    end

    return (x, gradient)
end

BFGS2 (generic function with 1 method)

In [4]:
f = (x1, x2) -> 100 * (x2 - x1^2)^2 + (1 - x1)^2
gf = (x1, x2) -> [-2 * (1-x1) - 400 * x1 * (- x1^2 + x2), 200 * (- x1^2 + x2)]
hf = (x1, x2) -> [(2 + 800 * x1^2 - 400 * (-x1^2 + x2)) (-400 * x1); (-400 * x1) 200]

function rosenbrock(x::Union{Nothing, AbstractVector})
    # optimal value in (0, [1, 1], hf(1, 1))
    if isnothing(x)
        return (f(-1, 1), [-1, 1], hf(-1, 1))
    end

    return (f(x[1], x[2]), gf(x[1], x[2]), hf(x[1], x[2]))
end

function sixhumpcamel(x::Union{Nothing, AbstractVecOrMat})
    if isnothing(x) # informative call
        v = -1.03162845349
        return (v, [1, 1], [0 0; 0 0])
    else
        v = ( 4 - 2.1 * x[1]^2 + x[1]^4 / 3 ) * x[1]^2 + x[1] * x[2] +
            4 * ( x[2]^2 - 1 ) * x[2]^2  # f(x)

        g = zeros(2)
        g[1] = 2 * x[1]^5 - (42 * x[1]^3) / 5 + 8 * x[1] + x[2]
        g[2] = 16 * x[2]^3 - 8 * x[2] + x[1]

        H = zeros(2, 2)
        H[1, 1] = 10 * x[1]^4 - ( 126 * x[1]^2 ) / 5 + 8
        H[2, 2] = 48 * x[2]^2 - 8
        H[2, 1] = 1
        H[1, 2] = H[2, 1]

        return (v, g, H)
    end
end

sixhumpcamel (generic function with 1 method)

In [13]:
BFGS2(rosenbrock)

([0.9999999823110854, 0.9999999651917256], [-2.6319958653672323e-7, 1.1391088072798539e-7])

In [14]:
BFGS2(sixhumpcamel)

([0.08984201171533876, -0.7126564021700074], [-9.948294188433238e-9, 1.254848300269451e-8])